<a href="https://colab.research.google.com/github/AntekPraktykant/3TablePivot/blob/master/Perplexity_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from datetime import datetime
import re

In [ ]:
# === Load Perplexity API key ===
def load_token(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read().strip()

PERPLEXITY_API_KEY = load_token("PERPLEXITY_API_KEY.txt")

# === Load Forum API key ===
def load_token(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read().strip()

FORUM_API_KEY   = load_token("FORUM_API_KEY.txt")
forum_url       = "https://forum.investopolis.pl/api/forums/posts"
auth            = (FORUM_API_KEY, "")       # Basic Auth
FORUM_AUTHOR_ID = 1                     # Twój ID użytkownika

In [ ]:
# === Pobranie newsów ===
today = datetime.now().strftime("%d.%m.%Y")
perp_url = "https://api.perplexity.ai/chat/completions"
perp_headers = {"Authorization": f"Bearer {PERPLEXITY_API_KEY}"}
perp_payload = {
    "model": "sonar",
    "messages": [
        {"role": "system", "content": "You are an expert on Warsaw Stock Exchange events."},
        {"role": "user",   "content": f"Podaj 5 najważniejszych wydarzeń z GPW z dnia {today}. Nie dodawaj komentarza. Potrzebuję po prostu 'gołego' newsa"}
    ],
    "search_domain_filter": ["gpw.pl", "biznes.onet.pl", "bankier.pl", "espiebi.pap.pl"],
}

response = requests.post(perp_url, headers=perp_headers, json=perp_payload)
response.raise_for_status()

# news_content trzymamy w zmiennej – będzie dostępna w kolejnej komórce
news_content = response.json()["choices"][0]["message"]["content"]
print("=== NEWSY Z PERPLEXITY ===")
print(news_content)

In [ ]:
# === Parsowanie Markdown → HTML ===
parts = re.split(r'(?=\d+\.\s)', news_content, flags=re.S)
items = []
for p in parts[1:]:
    m = re.match(r'(\d+)\.\s*(.+)', p, flags=re.S)
    if m:
        items.append(m.group(2).strip())

def md_to_html(txt):
    return re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', txt, flags=re.S)

# === Składamy finalne HTML z dynamicznym tytułem ===
html  = f"<p><strong>Puls GPW z {today}</strong></p>\n"
html += "<ol>\n"
for itm in items:
    html += f"  <li>{md_to_html(itm)}</li>\n"
html += "</ol>"

# === Usunięcie znacznikó źródeł [2] itp. ===
html = re.sub(r'\[\d+\]', '', html)

# === Wysyłka na forum ===
forum_payload = {
    "topic": 612,
    "author": FORUM_AUTHOR_ID,
    "post": html
}

resp = requests.post(forum_url, auth=auth, data=forum_payload)
if 200 <= resp.status_code < 300:
    data = resp.json()
    print(f"✅ Wysłano newsy na forum pomyślnie! ID posta: {data.get('id')}")
else:
    print(f"❌ Błąd podczas wysyłki: {resp.status_code} – {resp.text}")